Copyright 2021-2022 @ Shenzhen Bay Laboratory & Peking University & Huawei Technologies Co., Ltd

This code is a part of Cybertron package.

The Cybertron is open-source software based on the AI-framework:
MindSpore (https://www.mindspore.cn/)

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.

You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.

See the License for the specific language governing permissions and
limitations under the License.

Cybertron tutorial 07: Dataset with force

In [1]:
import time
import numpy as np
import mindspore as ms
from mindspore import nn
from mindspore import Tensor
from mindspore import dataset as ds
from mindspore import context
from mindspore.train import Model
from mindspore.train.callback import ModelCheckpoint, CheckpointConfig

from cybertron import Cybertron
from cybertron import MolCT
from cybertron import AtomwiseReadout
from cybertron.train import MAE, RMSE, MLoss, MSELoss
from cybertron.train import WithForceLossCell, WithForceEvalCell
from cybertron.train import TrainMonitor
from cybertron.train import TransformerLR

context.set_context(mode=context.GRAPH_MODE, device_target="GPU")

In [2]:
sys_name = 'dataset_ethanol_normed_'

train_file = sys_name + 'trainset_1024.npz'
valid_file = sys_name + 'validset_128.npz'

train_data = np.load(train_file)
valid_data = np.load(valid_file)

atom_types = Tensor(train_data['Z'], ms.int32)
scale = train_data['scale']
shift = train_data['shift']

In [3]:
mod = MolCT(
    cutoff=1,
    n_interaction=3,
    dim_feature=128,
    n_heads=8,
    max_cycles=1,
    fixed_cycles=True,
    length_unit='nm',
)

In [4]:
readout = AtomwiseReadout(mod, dim_output=1)

In [5]:
net = Cybertron(mod, readout=readout, atom_types=atom_types, length_unit='nm')
net.print_info()

Cybertron Engine, Ride-on!
--------------------------------------------------------------------------------
    Length unit: nm
    Input unit scale: 1
    Using fixed atom type index:
       Atom 0:     6
       Atom 1:     6
       Atom 2:     8
       Atom 3:     1
       Atom 4:     1
       Atom 5:     1
       Atom 6:     1
       Atom 7:     1
       Atom 8:     1
--------------------------------------------------------------------------------
    Deep molecular model:  MolCT
--------------------------------------------------------------------------------
       Length unit: nm
       Atom embedding size: 64
       Cutoff distance: 1.0 nm
       Radical basis function (RBF): LogGaussianBasis
          Minimum distance: 0.04 nm
          Maximum distance: 1.0 nm
          Reference distance: 1.0 nm
          Log Gaussian begin: -3.218876
          Log Gaussian end: 0.006724119
          Interval for log Gaussian: 0.0512
          Sigma for log gaussian: 0.3
          Number of ba

[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:31.672.697 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:260] CalMemBlockAllocSize] Device memory size [8781824] is smaller than minimum alloc size [10485760].
[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:31.672.945 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:260] CalMemBlockAllocSize] Device memory size [8781824] is smaller than minimum alloc size [10485760].
[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:31.673.035 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:260] CalMemBlockAllocSize] Device memory size [8781824] is smaller than minimum alloc size [10485760].


In [6]:
tot_params = 0
for i, param in enumerate(net.trainable_params()):
    tot_params += param.size
    print(i, param.name, param.shape)
print('Total parameters: ', tot_params)

0 model.atom_embedding.embedding_table (64, 128)
1 model.dis_filter.linear.weight (128, 64)
2 model.dis_filter.linear.bias (128,)
3 model.dis_filter.residual.nonlinear.mlp.0.weight (128, 128)
4 model.dis_filter.residual.nonlinear.mlp.0.bias (128,)
5 model.dis_filter.residual.nonlinear.mlp.1.weight (128, 128)
6 model.dis_filter.residual.nonlinear.mlp.1.bias (128,)
7 model.interactions.0.positional_embedding.norm.gamma (128,)
8 model.interactions.0.positional_embedding.norm.beta (128,)
9 model.interactions.0.positional_embedding.x2q.weight (128, 128)
10 model.interactions.0.positional_embedding.x2k.weight (128, 128)
11 model.interactions.0.positional_embedding.x2v.weight (128, 128)
12 model.interactions.0.multi_head_attention.output.weight (128, 128)
13 model.interactions.1.positional_embedding.norm.gamma (128,)
14 model.interactions.1.positional_embedding.norm.beta (128,)
15 model.interactions.1.positional_embedding.x2q.weight (128, 128)
16 model.interactions.1.positional_embedding.x2k.

In [7]:
n_epoch = 8
repeat_time = 1
batch_size = 32

In [8]:
ds_train = ds.NumpySlicesDataset(
    {'R': train_data['R'], 'F': train_data['F'], 'E': train_data['E']}, shuffle=True)
ds_train = ds_train.batch(batch_size)
ds_train = ds_train.repeat(repeat_time)

In [9]:
ds_valid = ds.NumpySlicesDataset(
    {'R': valid_data['R'], 'F': valid_data['F'], 'E': valid_data['E']}, shuffle=False)
ds_valid = ds_valid.batch(128)
ds_valid = ds_valid.repeat(1)

In [10]:
force_dis = train_data['avg_force_dis']
loss_fn = MSELoss(ratio_energy=1, ratio_forces=100, force_dis=force_dis)
loss_network = WithForceLossCell('RFE', net, loss_fn)
eval_network = WithForceEvalCell('RFE', net, loss_fn, scale=scale, shift=shift)

WithForceLossCell with input type: RFE
WithForceEvalCell with input type: RFE
   with scaleshift for training and evaluate dataset:
   Scale: [1.9488194]
   Shift: [-45185.31]
   Scaleshift mode: atomwise


In [11]:
lr = TransformerLR(learning_rate=1., warmup_steps=4000, dimension=128)
optim = nn.Adam(params=net.trainable_params(), learning_rate=lr)

In [12]:
energy_mae = 'EnergyMAE'
forces_mae = 'ForcesMAE'
forces_rmse = 'ForcesRMSE'
eval_loss = 'EvalLoss'
model = Model(loss_network, eval_network=eval_network, optimizer=optim,
              metrics={eval_loss: MLoss(), energy_mae: MAE([1, 2]), forces_mae: MAE([3, 4]),
                       forces_rmse: RMSE([3, 4], atom_aggregate='sum')})

In [13]:
outdir = 'Tutorial_C07'
outname = outdir + '_' + net.model_name
record_cb = TrainMonitor(model, outname, per_epoch=1, avg_steps=32,
                         directory=outdir, eval_dataset=ds_valid, best_ckpt_metrics=forces_rmse)

In [14]:
config_ck = CheckpointConfig(save_checkpoint_steps=32, keep_checkpoint_max=64, append_info=[net.hyper_param])
ckpoint_cb = ModelCheckpoint(prefix=outname, directory=outdir, config=config_ck)

In [15]:
print("Start training ...")
beg_time = time.time()
model.train(n_epoch, ds_train, callbacks=[
    record_cb, ckpoint_cb], dataset_sink_mode=False)
end_time = time.time()
used_time = end_time - beg_time
m, s = divmod(used_time, 60)
h, m = divmod(m, 60)
print("Training Fininshed!")
print("Training Time: %02d:%02d:%02d" % (h, m, s))

[WARNING] ME(4638:139730693121856,MainProcess):2022-08-15-14:48:31.843.899 [mindspore/train/model.py:1077] For TrainMonitor callback, {'begin', 'epoch_end', 'step_end'} methods may not be supported in later version, Use methods prefixed with 'on_train' or 'on_eval' instead when using customized callbacks.


Start training ...


[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:43.416.284 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:260] CalMemBlockAllocSize] Device memory size [8781824] is smaller than minimum alloc size [10485760].
[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:43.416.543 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:260] CalMemBlockAllocSize] Device memory size [8781824] is smaller than minimum alloc size [10485760].
[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:43.416.612 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:260] CalMemBlockAllocSize] Device memory size [8781824] is smaller than minimum alloc size [10485760].
[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:43.416.668 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:260] CalMemBlockAllocSize] Device memory size [8781824] is smaller than minimum alloc size [10485760].
[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:43.

Epoch: 1, Step: 32, Learning_rate: 1.0830951e-05, Last_Loss: 1.2025675, Avg_loss: 1.3432813882827759, EvalLoss: 1.3036696910858154, EnergyMAE: 12.695068359375, ForcesMAE: 807.6848958333334, ForcesRMSE: 1866.7657116580597


[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:50.377.520 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:260] CalMemBlockAllocSize] Device memory size [8781824] is smaller than minimum alloc size [10485760].
[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:50.377.675 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:260] CalMemBlockAllocSize] Device memory size [8781824] is smaller than minimum alloc size [10485760].
[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:50.377.753 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:260] CalMemBlockAllocSize] Device memory size [8781824] is smaller than minimum alloc size [10485760].
[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:50.377.835 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:260] CalMemBlockAllocSize] Device memory size [8781824] is smaller than minimum alloc size [10485760].
[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:50.

...

Epoch: 6, Step: 192, Learning_rate: 6.6732646e-05, Last_Loss: 0.09438597, Avg_loss: 0.14108049543574452, EvalLoss: 0.09790611267089844, EnergyMAE: 5.77783203125, ForcesMAE: 209.77772352430554, ForcesRMSE: 508.8093945673566


[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:56.141.025 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:260] CalMemBlockAllocSize] Device memory size [8781824] is smaller than minimum alloc size [10485760].
[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:56.141.094 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:260] CalMemBlockAllocSize] Device memory size [8781824] is smaller than minimum alloc size [10485760].
[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:56.141.117 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:260] CalMemBlockAllocSize] Device memory size [8781824] is smaller than minimum alloc size [10485760].
[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:56.187.530 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:260] CalMemBlockAllocSize] Device memory size [8781824] is smaller than minimum alloc size [10485760].
[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:56.

Epoch: 7, Step: 224, Learning_rate: 7.791298e-05, Last_Loss: 0.101164095, Avg_loss: 0.0990064712241292, EvalLoss: 0.07736483216285706, EnergyMAE: 8.016357421875, ForcesMAE: 184.38339572482639, ForcesRMSE: 446.8177449724017


[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:57.267.918 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:260] CalMemBlockAllocSize] Device memory size [8781824] is smaller than minimum alloc size [10485760].
[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:57.267.972 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:260] CalMemBlockAllocSize] Device memory size [8781824] is smaller than minimum alloc size [10485760].
[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:57.267.998 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:260] CalMemBlockAllocSize] Device memory size [8781824] is smaller than minimum alloc size [10485760].
[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:57.295.706 [mindspore/ccsrc/common/mem_reuse/mem_dynamic_allocator.cc:260] CalMemBlockAllocSize] Device memory size [8781824] is smaller than minimum alloc size [10485760].
[WARNING] PRE_ACT(4638,7f14ba885700,python):2022-08-15-14:48:57.

Epoch: 8, Step: 256, Learning_rate: 8.9093315e-05, Last_Loss: 0.07756487, Avg_loss: 0.07580070896074176, EvalLoss: 0.06250859797000885, EnergyMAE: 6.372802734375, ForcesMAE: 165.28597005208334, ForcesRMSE: 403.19811783070395
Training Fininshed!
Training Time: 00:00:26
